<a href="https://colab.research.google.com/github/yasinbastug/CommentToxicityBot/blob/main/reddit_bot_comment_toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 5.0 MB/s eta 0:00:00


In [4]:
import praw

# Reddit API credentials
client_id = "mvvM2FGj4gbIsdnLWbMQpg"
client_secret = "7XlXZtEzI3ZleDSmDeELme87m8WLXw"
user_agent = "bot:toxic:1.0"
username = "IsThisToxicBot"
password = "***********"

# Create a Reddit instance
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, username=username,
    password=password, user_agent=user_agent, check_for_async=False)


In [5]:
!pip install gradio_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [6]:
from gradio_client import Client




In [7]:
import json

In [21]:
# Split the input string by newlines and colons
def jsoner(result):

  pairs = [pair.split(":") for pair in result.split("\n")]
  pairs = pairs[:-1]

  # Remove leading/trailing spaces from keys and values
  labels = {key.strip(): value.strip() == "True" for key, value in pairs}

  # Convert the dictionary to JSON
  json_data = json.dumps(labels)

  return json_data


In [22]:
client = Client("https://yasinbastug-comment-toxicity.hf.space/")
def predict_json_data(comment):
  result = client.predict(
				comment,	# str  in 'comment' Textbox component
				api_name="/predict")
  return jsoner(result)




Loaded as API: https://yasinbastug-comment-toxicity.hf.space/ ✔


In [12]:
import prawcore
import time

In [ ]:
subreddit_name = "politicsturkey"

# Get the subreddit instance
subreddit = reddit.subreddit(subreddit_name)

# Stream and reply to mentions in the subreddit
# Stream and reply to mentions in the subreddit
new_line = '\n'
while True:
    try:
        for mention in reddit.inbox.stream():
            if isinstance(mention, praw.models.Comment) and mention.was_comment:
                reply_context = ""
                comment = mention.parent() # Get the parent comment and its context
                comment.refresh# Refresh to get the updated context
                context = comment.body
                results = json.loads(predict_json_data(context))
                for attribute, value in results.items():
                  if value:
                      reply_context += f"->{attribute}{new_line}"
                if (reply_context != ""):
                  reply_text = f"Attention! Your message may be against community rules for containing{new_line}{reply_context}Please be kind!"
                  comment.reply(reply_text)
                else:
                  comment.reply("I don't think this comment is")
                  print()

    except (prawcore.exceptions.RequestException, prawcore.exceptions.ServerError) as e:
        print(f"Encountered an error: {str(e)}")
        print("Retrying in 10 seconds...")
        time.sleep(10)  # Wait for 10 seconds before retrying

    except praw.exceptions.RedditAPIException as e:
        if "RATELIMIT" in str(e):
            print("Rate limited. Waiting for 6 minutes...")
            time.sleep(360)  # Wait for 6 minutes (360 seconds) before retrying
        else:
            raise e



Rate limited. Waiting for 6 minutes...
